In [16]:
# Importar librerias
import pandas as pd
import numpy as np

from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [17]:
#Importar los datasets
#url_embeddings_average = 'https://raw.githubusercontent.com/jjiguaran/text_mining/master/Data/embeddings_average.csv'
#url_embeddings_sum = 'https://raw.githubusercontent.com/jjiguaran/text_mining/master/Data/embeddings_sum.csv'

url_embeddings_average = 'C:/Users/dalessam/Downloads/text_mining/Data/embeddings_average.csv'
url_embeddings_sum = 'C:/Users/dalessam/Downloads/text_mining/Data/embeddings_sum.csv'

embeddings_average = pd.read_csv(url_embeddings_average)
embeddings_sum = pd.read_csv(url_embeddings_sum)

In [18]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average[pd.to_datetime(embeddings_average["Date"]) <= training_end])

### Embeddings Promedio

In [19]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average

In [20]:
# Se separa en train y test
x_train = data.drop([data.columns[0],"Label", "Date"], axis=1)[:num_training]
x_test = data.drop([data.columns[0],'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]

Se cambia el shape de train y test para el input:
https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

In [21]:
x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(1611, 1, 300)
reshape_x_train.shape

(1611, 1, 300)

In [22]:
x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(378, 1, 300)
reshape_x_test.shape

(378, 1, 300)

In [23]:
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint

In [27]:
n_units = 10

# AGREGAR SEED
model = Sequential()
model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# define the checkpoint
filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [28]:
# fit the model
model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
model.summary()

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
1611/1611 [==============================] - 1s 580us/step - loss: 0.6931 - accuracy: 0.5084

Epoch 00001: loss improved from inf to 0.69306, saving model to word2vec-01-0.6931.hdf5
Epoch 2/50
1611/1611 [==============================] - 0s 19us/step - loss: 0.6927 - accuracy: 0.5419

Epoch 00002: loss improved from 0.69306 to 0.69267, saving model to word2vec-02-0.6927.hdf5
Epoch 3/50
1611/1611 [==============================] - 0s 16us/step - loss: 0.6923 - accuracy: 0.5419

Epoch 00003: loss improved from 0.69267 to 0.69228, saving model to word2vec-03-0.6923.hdf5
Epoch 4/50
1611/1611 [==============================] - 0s 17us/step - loss: 0.6919 - accuracy: 0.5419

Epoch 00004: loss improved from 0.69228 to 0.69195, saving model to word2vec-04-0.6919.hdf5
Epoch 5/50
1611/1611 [==============================] - 0s 18us/step - loss: 0.6917 - accuracy: 0.5419

Epoch 00005: loss improved from 0.69195 to 0.69167, saving model to


Epoch 00048: loss did not improve from 0.68912
Epoch 49/50
1611/1611 [==============================] - 0s 20us/step - loss: 0.6894 - accuracy: 0.5419

Epoch 00049: loss did not improve from 0.68912
Epoch 50/50
1611/1611 [==============================] - 0s 19us/step - loss: 0.6894 - accuracy: 0.5419

Epoch 00050: loss did not improve from 0.68912
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1, 10)             12440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 10)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 13,291
Trainable p

In [29]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.507937


### Embeddings Suma

In [32]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_sum

In [33]:
# Se separa en train y test
x_train = data.drop([data.columns[0],"Label", "Date"], axis=1)[:num_training]
x_test = data.drop([data.columns[0],'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]

In [34]:
n_units = 5

model = Sequential()
model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(n_units, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
#binary_crossentropy
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# define the checkpoint
filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [35]:
x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(1611, 1, 300)
reshape_x_train.shape

(1611, 1, 300)

In [36]:
x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(378, 1, 300)
reshape_x_test.shape

(378, 1, 300)

In [37]:
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint

In [38]:
# fit the model
model.fit(reshape_x_train, y_train, epochs=50, batch_size=100, callbacks=callbacks_list)
model.summary()

Epoch 1/50
1611/1611 [==============================] - 1s 516us/step - loss: 0.2487 - accuracy: 0.5400

Epoch 00001: loss improved from inf to 0.24871, saving model to word2vec-01-0.2487.hdf5
Epoch 2/50
1611/1611 [==============================] - 0s 34us/step - loss: 0.2490 - accuracy: 0.5425

Epoch 00002: loss did not improve from 0.24871
Epoch 3/50
1611/1611 [==============================] - 0s 36us/step - loss: 0.2489 - accuracy: 0.5400

Epoch 00003: loss did not improve from 0.24871
Epoch 4/50
1611/1611 [==============================] - 0s 37us/step - loss: 0.2483 - accuracy: 0.5419

Epoch 00004: loss improved from 0.24871 to 0.24830, saving model to word2vec-04-0.2483.hdf5
Epoch 5/50
1611/1611 [==============================] - 0s 40us/step - loss: 0.2478 - accuracy: 0.5419

Epoch 00005: loss improved from 0.24830 to 0.24780, saving model to word2vec-05-0.2478.hdf5
Epoch 6/50
1611/1611 [==============================] - 0s 39us/step - loss: 0.2482 - accuracy: 0.5419

Epoch 000

In [39]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 50.793654
